# News Hour - Part III

In the previous notebooks, we got the data set up (Part I) and estimated a preliminary version of the viewership model (Part II). Now, what we want to do is:

1. Set up the log-likelihood for our pricing model, and
2. Estimate the price model, seperately, to get starting values, and just to make sure it is working correctly. 

Both are time-consuming, and to save time and trouble, we use the method described in [this paper](http://papers.ssrn.com/sol3/papers.cfm?abstract_id=420371), which is also described in [this paper](http://www.stata-journal.com/article.html?article=st0354). 

To estimate things using this method, you can read the latter paper above, and should download and install the `AMCMC` `Stata` module from `SSC`. It can be found [here](https://ideas.repec.org/c/boc/bocode/s457613.html).  

Usual beginning steps:

In [1]:
import ipystata
import os

CWD = os.getcwd()
print(CWD)

C:\Users\Matthew Baker\Documents\GitHub\NewsHour


In [3]:
%%stata
clear all
cd "C:\Users\Matthew Baker\Documents\GitHub\NewsHour"


C:\Users\Matthew Baker\Documents\GitHub\NewsHour


Get our data (which we assembled in Part I) - it is an averaged data set that should be ready to use. Also, be careful that the data is sorted correctly...

In [4]:
%%stata
use "AveragedDataDyno.dta", clear
set more off
sort market stationid mt

We need to generate a few variables to help in fitting our model. For one, we need to create total viewership, and we also need total viewership interacted with each type of programming. Since we are estimating a price model, we keep only observations for which the log of price-per-second is not missing...

In [5]:
%%stata
gen lnview=ln(ACS_HH*si)
gen lnviewl=lnews*lnview
gen lnviewo=otherl*lnview
gen lnviewn=nnews*lnview

keep if lnpps!=.


(95994 observations deleted)


## Preliminary Model
To get our feet wet, let's just estimate a two-way random effects model of prices (analogous to our viewership two-way random effects). This will also give us starting values for maximum likelihood. So, after estimation, I will store all the data and then pass it into mata. 

In [6]:
%%stata
a2reg lnpps lnviewl lnviewo lnviewn lnews otherl l_ACS_HH, individual(stationid) unit(mt) 

mat sdmodp=e(rmse)
mat sdstap=e(sdind)
mat bop=e(b)
mat sdmarp=e(sdunit)
mat alphap=e(constant)

mata:
    st_view(ma=.,.,"market")
    st_view(id=.,.,"stationid")
    st_view(mt=.,.,"mt")	

    bop=st_matrix("bop")
    
    sdmarp=st_matrix("sdmarp")
    sdstap=st_matrix("sdstap")
    sdmodp=st_matrix("sdmodp")
    alphap=st_matrix("alphap")	
    
    bop=bop,alphap
end


> d) unit(mt) 
3738 observations, 6 covariates, 623 individuals, 606 units, 3738 cells
Beginning Iterations
Starting Conjugate Gradient Algorithm
Iteration 0, norm of residual 232.152793, relative error 1
Iteration 1, norm of residual 28.7064917, relative error .123653441
Iteration 2, norm of residual 8.79840461, relative error .037899198
Iteration 3, norm of residual 3.12485817, relative error .013460351
Iteration 4, norm of residual 2.25898131, relative error .00973058
Iteration 5, norm of residual 3.4294973, relative error .014772587
Iteration 6, norm of residual 1.20489851, relative error .00519011
Iteration 7, norm of residual 1.257377, relative error .005416161
Iteration 8, norm of residual 1.42254447, relative error .006127622
Iteration 9, norm of residual 1.00250698, relative error .004318307
Iteration 10, norm of residual .908225852, relative error .00391219
Iteration 11, norm of residual .479498844, relative error .002065445
Iteration 12, norm of residual .282285386, relativ

So, we have a preliminary model with some preliminary parameter estimates. Let's make a price likelihood function and then estimate that out to get starting values. Again following Baltagi, we write down the two-way random effects likelihood and loop over the panels. Once again, the random effects are at the station level, and the market-time level. 

In [7]:
%%stata --mata
    void logLikelihoodPrice(M,todo,b,lnf,g,H)
    {
        real scalar nu_l,nu_o,nu_n,gamma_l,gamma_o,
            sdsta,sdmar,sdmod,i,T,N,lam1,
            lam2,lam3,lam4,phi22,phi32

        real matrix X,mt,id,mtp,idp,yp,Xp,Jn,Jt,En,Et,
            Q1,Q2,Q3,Q4,OmegaInv,beta

        nu_l =moptimize_util_xb(M,b,1)
        nu_o =moptimize_util_xb(M,b,2)
        nu_n =moptimize_util_xb(M,b,3)
        gamma_l=moptimize_util_xb(M,b,4)
        gamma_o=moptimize_util_xb(M,b,5)
        omega=moptimize_util_xb(M,b,6)
        sdsta=exp(moptimize_util_xb(M,b,7))
        sdmar=exp(moptimize_util_xb(M,b,8))
        sdmod=exp(moptimize_util_xb(M,b,9))
        alpha=moptimize_util_xb(M,b,10)

        y      =moptimize_util_depvar(M,1)
        lnews  =moptimize_util_depvar(M,2)
        otherl =moptimize_util_depvar(M,3)
        nnews  =moptimize_util_depvar(M,4)
        lnview =moptimize_util_depvar(M,5)
        l_ACS_HH=moptimize_util_depvar(M,6)

        id=moptimize_util_userinfo(M,1)
        mt=moptimize_util_userinfo(M,2)
        m =moptimize_util_userinfo(M,3)

        beta=nu_l,nu_o,nu_n,gamma_l,gamma_o,omega,alpha
        X=lnews:*lnview,otherl:*lnview,nnews:*lnview,
            lnews,otherl,l_ACS_HH,J(rows(lnews),1,1)

        lnf=J(rows(m),1,.)
        for (i=1;i<=rows(m);i++) {
            mtp=panelsubmatrix(mt,i,m)
            idp=panelsubmatrix(id,i,m)
            T=rows(uniqrows(mtp))
            N=rows(uniqrows(idp))
            yp=panelsubmatrix(y,i,m)
            Xp=panelsubmatrix(X,i,m)

            lam1=sdmod^2
            lam2=T*sdsta^2+sdmod^2
            lam3=N*sdmar^2+sdmod^2
            lam4=T*sdsta^2+N*sdmar^2+sdmod^2
            phi22=sdmod^2/lam2
            phi32=sdmod^2/lam3
            phi42=sdmod^2/lam4

            Jn=J(N,N,1/N)
            Jt=J(T,T,1/T)
            En=I(N)-Jn
            Et=I(T)-Jt

            Q1=En#Et
            Q2=En#Jt
            Q3=Jn#Et
            Q4=Jn#Jt
            OmegaInv=(Q1/lam1+Q2/lam2+Q3/lam3+Q4/lam4)
            lnDetOmega=-2*N*T*ln(sdmod)+(N-1)*ln(phi22)+(T-1)*ln(phi32)+ln(phi42)
            lnf[i]=1/2*lnDetOmega-1/2*(yp-Xp*beta')'*OmegaInv*(yp-Xp*beta')
        }
        if (hasmissing(lnf)) lnf=.
        else lnf=colsum(lnf)
    }
                                                               


:     void logLikelihoodPrice(M,todo,b,lnf,g,H)
>     {
>         real scalar nu_l,nu_o,nu_n,gamma_l,gamma_o,
>             sdsta,sdmar,sdmod,i,T,N,lam1,
>             lam2,lam3,lam4,phi22,phi32
> 
>         real matrix X,mt,id,mtp,idp,yp,Xp,Jn,Jt,En,Et,
>             Q1,Q2,Q3,Q4,OmegaInv,beta
> 
>         nu_l =moptimize_util_xb(M,b,1)
>         nu_o =moptimize_util_xb(M,b,2)
>         nu_n =moptimize_util_xb(M,b,3)
>         gamma_l=moptimize_util_xb(M,b,4)
>         gamma_o=moptimize_util_xb(M,b,5)
>         omega=moptimize_util_xb(M,b,6)
>         sdsta=exp(moptimize_util_xb(M,b,7))
>         sdmar=exp(moptimize_util_xb(M,b,8))
>         sdmod=exp(moptimize_util_xb(M,b,9))
>         alpha=moptimize_util_xb(M,b,10)
> 
>         y      =moptimize_util_depvar(M,1)
>         lnews  =moptimize_util_depvar(M,2)
>         otherl =moptimize_util_depvar(M,3)
>         nnews  =moptimize_util_depvar(M,4)
>         lnview =moptimize_util_depvar(M,5)
>         l_ACS_HH=moptimize_util_depvar(M,

As is usual, we have to first just run the function once, set some parameters for the MCMC run, get initial values, etc. etc. So, here is a preliminary run to achieve this.

In [8]:
%%stata --mata
    Z=moptimize_init()
    moptimize_init_trace_dots(Z,"on")
    moptimize_init_trace_params(Z,"on")
    moptimize_init_evaluator(Z,&logLikelihoodPrice())
    moptimize_init_evaluatortype(Z,"d0")
    moptimize_init_which(Z,"max")

    moptimize_init_eq_indepvars(Z,1,"")	/* nu_l */
    moptimize_init_eq_indepvars(Z,2,"")	/* nu_o */
    moptimize_init_eq_indepvars(Z,3,"")	/* nu_n */
    moptimize_init_eq_indepvars(Z,4,"")     /* gamma_l */
    moptimize_init_eq_indepvars(Z,5,"")     /* gamma_o */
    moptimize_init_eq_indepvars(Z,6,"")	/* omega_p */
    moptimize_init_eq_indepvars(Z,7,"")     /* ln(sdsta) */
    moptimize_init_eq_indepvars(Z,8,"")     /* ln(sdmar) */
    moptimize_init_eq_indepvars(Z,9,"")	/* ln(mod)   */
    moptimize_init_eq_indepvars(Z,10,"")	/* alphap    */

    moptimize_init_depvar(Z,1,"lnpps")
    moptimize_init_depvar(Z,2,"lnews")
    moptimize_init_depvar(Z,3,"otherl")
    moptimize_init_depvar(Z,4,"nnews")
    moptimize_init_depvar(Z,5,"lnview")
    moptimize_init_depvar(Z,6,"l_ACS_HH")
    
    m=panelsetup(ma,1)
    moptimize_init_userinfo(Z,1,id)
    moptimize_init_userinfo(Z,2,mt)
    moptimize_init_userinfo(Z,3,m)
    moptimize_evaluate(Z)
    

Mata output:

:     Z=moptimize_init()

:     moptimize_init_trace_dots(Z,"on")

:     moptimize_init_trace_params(Z,"on")

:     moptimize_init_evaluator(Z,&logLikelihoodPrice())

:     moptimize_init_evaluatortype(Z,"d0")

:     moptimize_init_which(Z,"max")

:     moptimize_init_eq_indepvars(Z,2,"")/* nu_o */

:     moptimize_init_eq_indepvars(Z,3,"")/* nu_n */

:     moptimize_init_eq_indepvars(Z,4,"")     /* gamma_l */

:     moptimize_init_eq_indepvars(Z,5,"")     /* gamma_o */

:     moptimize_init_eq_indepvars(Z,6,"")/* omega_p */

:     moptimize_init_eq_indepvars(Z,7,"")     /* ln(sdsta) */

:     moptimize_init_eq_indepvars(Z,8,"")     /* ln(sdmar) */

:     moptimize_init_eq_indepvars(Z,9,"")/* ln(mod)   */

:     moptimize_init_eq_indepvars(Z,10,"")/* alphap    */

:     moptimize_init_depvar(Z,2,"lnews")

:     moptimize_init_depvar(Z,3,"otherl")

:     moptimize_init_depvar(Z,4,"nnews")

:     moptimize_init_depvar(Z,5,"lnview")

:     moptimize_init_depvar(Z,6,"l_ACS_HH

That having been done, I now can set up a run and execute it. Note: it might be good to have a formal schematic depicting the relationship between Stata parameters and model parameters. 

Let's just make sure everything is in working order to start...

In [9]:
%%stata --mata
    boinit=st_matrix("bop"),sdmarp,sdstap,sdmodp,alphap
    alginfo="mwg","d0","moptimize"
    bp_start=amcmc(alginfo,&logLikelihoodPrice(),boinit,I(10),80,20,2/3,.4,arate=.,vals=.,lambda=.,.,Z,"noisy")
    bop=bp_start[rows(bp_start),]

Mata output:

:     boinit=st_matrix("bop"),sdmarp,sdstap,sdmodp,alphap

:     alginfo="mwg","d0","moptimize"

:     bp_start=amcmc(alginfo,&logLikelihoodPrice(),boinit,I(10),80,20,2/3,.4,ara
> te=.,vals=.,lambda=.,.,Z,"noisy")
................................................. 50: f(x) = 1005.753
................................................. 100: f(x) = 1069.68152
................................................. 150: f(x) = 1079.39236
................................................. 200: f(x) = 1081.51704
................................................. 250: f(x) = 1081.95645
................................................. 300: f(x) = 1080.11347
................................................. 350: f(x) = 1080.62074
................................................. 400: f(x) = 1079.8671
................................................. 450: f(x) = 1087.22659
................................................. 500: f(x) = 1087.24639
..............................................

I will now try a continuation run and see how that goes...but this time we will try to use the standard optimizer, to get an actual maximum...and then maybe try getting some draws in order. 

In [9]:
%%stata
mata:
    Z1=moptimize_init()
    moptimize_init_trace_dots(Z1,"on")
    moptimize_init_trace_params(Z1,"on")
    moptimize_init_evaluator(Z1,&logLikelihoodPrice())
    moptimize_init_evaluatortype(Z1,"d0")
    moptimize_init_which(Z1,"max")

    moptimize_init_eq_indepvars(Z1,1,"")    /* nu_l */
    moptimize_init_eq_indepvars(Z1,2,"")    /* nu_o */
    moptimize_init_eq_indepvars(Z1,3,"")    /* nu_n */
    moptimize_init_eq_indepvars(Z1,4,"")     /* eta_l */
    moptimize_init_eq_indepvars(Z1,5,"")     /* eta_o */
    moptimize_init_eq_indepvars(Z1,6,"")     /* eta_n */
    moptimize_init_eq_indepvars(Z1,7,"")     /* omega */
    moptimize_init_eq_indepvars(Z1,8,"")     /* ln(sdsta) */
    moptimize_init_eq_indepvars(Z1,9,"")     /* ln(sdmar) */
    moptimize_init_eq_indepvars(Z1,10,"")    /* ln(sdmod) */
    
    moptimize_init_depvar(Z1,1,"lnpps")
    moptimize_init_depvar(Z1,2,"lnews")
    moptimize_init_depvar(Z1,3,"otherl")
    moptimize_init_depvar(Z1,4,"nnews")
    moptimize_init_depvar(Z1,5,"lnview")
    moptimize_init_depvar(Z1,6,"l_ACS_HH")

    m=panelsetup(ma,1)
    moptimize_init_userinfo(Z1,1,id)
    moptimize_init_userinfo(Z1,2,mt)
    moptimize_init_userinfo(Z1,3,m)
    
    moptimize_init_eq_coefs(Z1,1,bop[1])
    moptimize_init_eq_coefs(Z1,2,bop[2])
    moptimize_init_eq_coefs(Z1,3,bop[3])
    moptimize_init_eq_coefs(Z1,4,bop[4])
    
    moptimize_init_eq_coefs(Z1,5,bop[5])
    moptimize_init_eq_coefs(Z1,6,bop[6])
    moptimize_init_eq_coefs(Z1,7,bop[7])
    moptimize_init_eq_coefs(Z1,8,bop[8])
    moptimize_init_eq_coefs(Z1,9,bop[9])
    moptimize_init_eq_coefs(Z1,10,bop[10])
    moptimize(Z1)
    bpo=moptimize_result_coefs(Z1)
end


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     Z1=moptimize_init()

:     moptimize_init_trace_dots(Z1,"on")

:     moptimize_init_trace_params(Z1,"on")

:     moptimize_init_evaluator(Z1,&logLikelihoodPrice())

:     moptimize_init_evaluatortype(Z1,"d0")

:     moptimize_init_which(Z1,"max")

:     moptimize_init_eq_indepvars(Z1,2,"")    /* nu_o */

:     moptimize_init_eq_indepvars(Z1,3,"")    /* nu_n */

:     moptimize_init_eq_indepvars(Z1,4,"")     /* eta_l */

:     moptimize_init_eq_indepvars(Z1,5,"")     /* eta_o */

:     moptimize_init_eq_indepvars(Z1,6,"")     /* eta_n */

:     moptimize_init_eq_indepvars(Z1,7,"")     /* omega */

:     moptimize_init_eq_indepvars(Z1,8,"")     /* ln(sdsta) */

:     moptimize_init_eq_indepvars(Z1,9,"")     /* ln(sdmar) */

:     moptimize_init_eq_indepvars(Z1,10,"")    /* ln(sdmod) */

:     
:     moptimize_init_depvar(Z1,1,"lnpps")

:     mo

So, we see that the results are a little bit different than `a2reg` - but not much. Anyways, I'm going to redraw values again just to be thorough, from this point on. Then, I will save these results for further use later. Here goes:

In [10]:
%%stata
mata: 
    b_start=amcmc(alginfo,&logLikelihoodPrice(),bpo,diag(abs(bpo)/4),220,20,1,.4,arate=.,vals=.,lambda=.,.,Z,"noisy")
    bpo=b_start[rows(b_start),]
    drawsbpo=b_start

    mata matsave betaPDynoStarts bpo drawsbpo, replace
end


------------------------------------------------- mata (type end to exit) ----------------------------------------------------------------
:     b_start=amcmc(alginfo,&logLikelihoodPrice(),bpo,diag(abs(bpo)/4),220,20,1,.4,arate=.,vals=.,lambda=.,.,Z,"noisy")
................................................. 50: f(x) = 1429.8495
................................................. 100: f(x) = 1429.57161
................................................. 150: f(x) = 1429.27417
................................................. 200: f(x) = 1426.95594
................................................. 250: f(x) = 1427.18241
................................................. 300: f(x) = 1427.32213
................................................. 350: f(x) = 1427.41909
................................................. 400: f(x) = 1425.53824
................................................. 450: f(x) = 1428.61552
................................................. 500: f(x) = 1429.88137
............

While I am not sure it is necessary, we can save this data for later use in matrix form:

In [10]:
%%stata --mata
    st_view(lnppst=.,.,"lnpps")
    st_view(lnewst=.,.,"lnews")
    st_view(otherlt=.,.,"otherl")
    st_view(nnewst=.,.,"nnews")
    st_view(lnviewt=.,.,"lnview")
    st_view(l_ACS_HHt=.,.,"l_ACS_HH")

    mat=m
    idt=id
    mtt=mt   
    
    mata matsave MataPriceData lnppst lnewst otherlt nnewst lnviewt l_ACS_HHt mat idt mtt, replace
    

Mata output:

:     st_view(lnppst=.,.,"lnpps")

:     st_view(lnewst=.,.,"lnews")

:     st_view(otherlt=.,.,"otherl")

:     st_view(nnewst=.,.,"nnews")

:     st_view(lnviewt=.,.,"lnview")

:     st_view(l_ACS_HHt=.,.,"l_ACS_HH")

:     idt=id

:     mtt=mt   

:     
:     mata matsave MataPriceData lnppst lnewst otherlt nnewst lnviewt l_ACS_HHt 
> mat idt mtt, replace
(saving idt[3738,1], l_ACS_HHt[3738,1], lnewst[3738,1], lnppst[3738,1],
 lnviewt[3738,1], mat[101,2], mtt[3738,1], nnewst[3738,1], otherlt[3738,1])
file MataPriceData.mmat saved

:     


** Note: ** sometimes the above craps out if one hasn't run the whole thing through, or if one starts another notebook in the middle of things. But it will run on its own!